In [1]:
import scipy
import scipy.sparse, scipy.stats
import numpy
import os
import re
import multiprocessing
import subprocess
import pickle
import matplotlib.pyplot
import time
import statistics
import sklearn
import statsmodels , statsmodels.api
import itertools

In [2]:
def moving_integration (values, window):
    weights = numpy.repeat(1.0, window)
    sma = numpy.convolve(values, weights, 'same')
    return sma
def moving_average (values, window):
    weights = numpy.repeat(1.0, window)/window
    sma = numpy.convolve(values, weights, 'same')
    return sma

In [11]:
with open("./testdata/peaks_tests.pi" ,"rb") as picklein:
    smoothed_diagonal,quick_peaks = pickle.load(picklein)

In [9]:
#needed to load the variables
with open("./testdata/variables.pi","rb") as picklefile:
    frag_index,frag_prop,frag_amount,valid_chroms,chroms_offsets = pickle.load(picklefile)

In [5]:
print(sum([1 if x else 0 for x in quick_peaks]))
print(frag_amount["chr1"])

60176
584662


In [6]:
import re
print(len(re.findall(r'(0+|1+)',"".join(["1" if x else "0" for x in quick_peaks])))/0.16)

63931.25


In [7]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.hist(smoothed_diagonal,range=(0,500), bins=500)
fig.show()

In [6]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.hist(list(itertools.compress(smoothed_diagonal, [not i for i in quick_peaks])),range=(0,500), bins=500)
ax.hist(scipy.stats.poisson.rvs(numpy.mean(smoothed_diagonal), size = 100000),range=(0,500), bins=500)
ax.hist(scipy.stats.poisson.rvs(10, size = 200000),range=(0,500), bins=500)
fig.show()

In [14]:
lengths = [x[3] for x in frag_prop][0:584662]
lengths = moving_integration(lengths, 4)
compress_lengths = list(itertools.compress(lengths, [not i for i in quick_peaks]))
compress_diagonal = list(itertools.compress(smoothed_diagonal, [not i for i in quick_peaks]))
#compress_diagonal = [x for x in compress_diagonal if x>0]

In [14]:

%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.hist(compress_diagonal,range=(0,500), bins=500)
#ax.hist(scipy.stats.binom.rvs(numpy.mean(smoothed_diagonal), size = 100000),range=(0,500), bins=500)
ax.hist(scipy.stats.nbinom.rvs(1.78,0.071, size = 584663),range=(0,500), bins=500)
fig.show()

In [15]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()

ax.hist2d(compress_lengths,compress_diagonal,(700,100),range=((0,7000),(0,100)),cmap=matplotlib.pyplot.cm.jet,cmax=90)

fig.show()

In [7]:
a = [1,1,1,5,1,1]
b = [|1,1,1,6,1,1]
moving_integration(a,2)
#no alignment neccesary??? site 3 needs the integration on fragment 3

array([1., 2., 2., 6., 6., 2.])

In [15]:
y = statsmodels.api.nonparametric.lowess(compress_diagonal[:30000], compress_lengths[:30000],return_sorted=False,frac=0.4,delta=3.0)
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.scatter(compress_lengths[:30000],y)
fig.show()

In [1]:
#add some correction for background around like 100kb. 10kb seems hard especially because some fragments are 10 kb

In [7]:
nbinom_data = statsmodels.api.NegativeBinomial(compress_diagonal,numpy.ones(len(compress_diagonal)))

In [8]:
nb = nbinom_data.fit()

Optimization terminated successfully.
         Current function value: 4.167043
         Iterations: 5
         Function evaluations: 6
         Gradient evaluations: 6


In [10]:
blabla =nb.summary()
#const is ln(mean)
#alpha is 1/r
#to get p it's mean = (1-p)*r/p

In [12]:
a = [nbinom_data.pdf(x) for x in range(500)]
matplotlib.pyplot.plot(a)

NotImplementedError: 

In [27]:
numpy.percentile(scipy.stats.nbinom.rvs(2.22,0.075, size = 584663),95)

64.0

In [6]:
numpy.percentile(lengths,99)

NameError: name 'lengths' is not defined

In [40]:
nb.params

array([4.09604928, 1.6148868 ])

In [3]:
scipy.stats.nbinom.sf(15, 2,0.2) + scipy.stats.nbinom.pmf(15 , 2,0.2)

0.14073748835532815

In [7]:
scipy.stats.nbinom_test(2,15,0.2,alternative='greater')

AttributeError: module 'scipy.stats' has no attribute 'nbinom_test'

In [15]:
%matplotlib qt

fig = matplotlib.pyplot.figure()
ax = fig.add_subplot(111)
ax.plot(numpy.array(numpy.loadtxt("./testdata/sorted_peakscounts")).cumsum())

In [15]:
####section for quantifying how good the peak caller is

%matplotlib qt

fig = matplotlib.pyplot.figure()
ax = fig.add_subplot(111)

#  bedtools annotate -counts -i peaks_10percent_GM12878.bed -files GM12878_replicated_peaks.bed | sort -k6,6 -r -g | cut -f7 > GM12878_sorted_peaks_counts.txt


ax.plot(numpy.array(numpy.loadtxt("./testdata/GM12878_sorted_peaks_counts.txt")).cumsum())

#for hichipper

# wc -l hichipper peaks to get the number of peaks for that level
# bedtools annotate -counts -i peaks_hichipper_GM12878.bed -files GM12878_replicated_peaks.bed | cut -f4 | paste -s -d+ | bc
#0.05
ax.scatter(69524,47273)

#0.01
ax.scatter(69605,47324)

#0.002
ax.scatter(69529,47288)


In [16]:
# bedtools annotate -counts -i peaks_full10percent.bed -files T_helper_replicated_peaks.bed | sort -k5,5 -r -g | cut -f6 > Tcells_sorted_peaks_counts.txt

%matplotlib qt

fig = matplotlib.pyplot.figure()
ax = fig.add_subplot(111)


ax.plot(numpy.array(numpy.loadtxt("./testdata/Tcells_sorted_peaks_counts.txt")).cumsum())

# bedtools annotate -counts -i mumbach_27ac.anchors.bed -files T_helper_replicated_peaks.bed | cut -f4 - | paste -s -d+ | bc
#wc -l mumba to get number of peaks
# bedtools slop -i mumbach_27ac.anchors.bed -b 1000 -g hg38.genome.txt | bedtools merge | bedtools annotate -counts -i - -files T_helper_replicated_peaks.bed | cut -f4 - | paste -s -d+ | bc

#0.05
ax.scatter(41530,32969)

#0.01
ax.scatter(41689,32978)

#0.002
ax.scatter(41489,32963)

